In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import shutil
import pandas as pd

def organize_images_by_category(train_csv_path, images_folder_path, output_base_path):
    # Load the CSV file
    df = pd.read_csv(train_csv_path)

    # Create a dictionary to map categories to folder names
    categories = {
        "Women Tops & Tunics": "Women_Tops_&_Tunics",
        "Women Tshirts": "Women_Tshirts",
        "Sarees": "Sarees",
        "Men Tshirts": "Men_Tshirts",
        "Kurtis": "Kurtis"
    }

    # Create folders for each category in /kaggle/working
    for category in categories.values():
        folder_path = os.path.join(output_base_path, category)
        os.makedirs(folder_path, exist_ok=True)

    # Iterate over the rows in the CSV and copy images to corresponding category folders
    for _, row in df.iterrows():
        image_id = str(row['id']).zfill(6)  # Convert ID to 6-digit string with leading zeros
        image_name = f"{image_id}.jpg"
        category = row['Category']

        if category in categories:
            # Get the source image path
            src_image_path = os.path.join(images_folder_path, image_name)
            
            # Get the destination folder for the category
            dest_folder = os.path.join(output_base_path, categories[category])
            
            # Get the destination image path
            dest_image_path = os.path.join(dest_folder, image_name)

            # Copy the image to the destination folder
            if os.path.exists(src_image_path):
                shutil.copy(src_image_path, dest_image_path)  # Copy instead of move
                #print(f"Copied {image_name} to {categories[category]} folder.")
            else:
                print(f"Image {image_name} not found in {images_folder_path}.")

if __name__ == "__main__":
    # Path to the CSV file and images folder
    train_csv_path = "/kaggle/input/visual-taxonomy/test.csv"  # Update with actual path to train.csv
    images_folder_path = "/kaggle/input/visual-taxonomy/test_images"  # Update with actual path to train_images
    output_base_path = "/kaggle/working/sorted_images"  # Writable directory

    # Organize the images by category
    organize_images_by_category(train_csv_path, images_folder_path, output_base_path)

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

def plot_image_if_exists(image_folder, image_name):
    # Construct the full path to the image
    image_path = os.path.join(image_folder, image_name)
    
    # Check if the image exists
    if os.path.exists(image_path):
        # Load the image
        image = Image.open(image_path)
        
        # Plot the image
        plt.imshow(image)
        plt.axis('off')  # Hide axis
        plt.title(f"Image: {image_name}")  # Set title
        plt.show()  # Display the image
    else:
        print(f"Image '{image_name}' does not exist in the folder '{image_folder}'.")

# Example usage
image_folder = "/kaggle/working/sorted_images/Men_Tshirts"  # Replace with your folder path
image_name = "/kaggle/input/visual-taxonomy/test_images/000001.jpg"  # Replace with the image name you want to check

plot_image_if_exists(image_folder, image_name)


In [ ]:
from tensorflow.keras.models import load_model

# Load your model (replace 'your_model.h5' with the actual model path)
model = load_model('/kaggle/input/modelm/model_checkpoint.keras')

# Verify that the model has been loaded
print("Model loaded successfully.")


In [3]:
import tensorflow as tf
from keras.saving import register_keras_serializable

@register_keras_serializable()
def custom_loss_function(y_true, y_pred):
    """
    Custom loss function to ignore the last class (dummy) during loss calculation.
    """
    # Exclude the last class (dummy) from the loss calculation
    y_true = tf.cast(y_true[:, :-1], tf.float32)  # Exclude last class
    y_pred = y_pred[:, :-1]  # Exclude last class
    loss = tf.keras.backend.categorical_crossentropy(y_true, y_pred)  # Use tf.keras.backend
    return loss


In [5]:
from tensorflow.keras.models import load_model

# Load your model (replace 'your_model.h5' with the actual model path)
model = load_model('/kaggle/input/model-bettercorr-s/model_checkpoint.keras')
#model2 = load_model('/kaggle/input/model-ensemble-m/model_checkpoint.keras')
#model3 = load_model('/kaggle/input/model-inception-m/model_checkpoint.keras')


# Verify that the model has been loaded
print("Model loaded successfully.")


Model loaded successfully.


In [ ]:
# Print the model summary to see its architecture
model.summary()

# Check the input shape of the model
input_shape = model.input_shape
print(f"Expected input shape: {input_shape}")


In [ ]:
from PIL import Image
import numpy as np

def preprocess_image(image_path):
    # Load the image
    image = Image.open(image_path)
    
    # Resize the image to (198, 198)
    image = image.resize((198, 198))
    
    # Convert the image to a numpy array and normalize pixel values
    image_array = np.array(image) / 255.0
    
    return image_array

# Example usage
image_path = '/kaggle/input/visual-taxonomy/test_images/000000.jpg'  # Replace with your image path
processed_image = preprocess_image(image_path)
print(processed_image.shape)  # Should be (198, 198, 3) for RGB images


In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image

def preprocess_image(image_path):
    # Load the image
    image = Image.open(image_path)
    
    # Resize the image to (198, 198)
    image = image.resize((198, 198))
    
    # Convert the image to a numpy array and normalize pixel values
    image_array = np.array(image) / 255.0
    
    # Add a batch dimension (1, 198, 198, 3)
    image_array = np.expand_dims(image_array, axis=0)
    
    return image_array

# Load your model
model = load_model('/kaggle/input/modelm/model_checkpoint.keras')  # Replace with your model path

# Predict on a new image
image_path = '/kaggle/input/visual-taxonomy/test_images/000000.jpg'  # Replace with your image path
processed_image = preprocess_image(image_path)

# Make predictions
predictions = model.predict(processed_image)

# Print the raw predictions
print(f"Raw predictions for {image_path}: {predictions}")


In [ ]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array

# Load and preprocess the image
def preprocess_image(image_path):
    # Load the image with target size (198, 198)
    img = load_img(image_path, target_size=(198, 198))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
    return np.expand_dims(img_array, axis=0)  # Add batch dimension

# Load your model (assuming model is already defined and compiled)
# model = ... # Your model loading code here

# Example image path
image_path = "/kaggle/input/visual-taxonomy/test_images/000000.jpg"

# Preprocess the image
preprocessed_image = preprocess_image(image_path)

# Get raw predictions from the model
raw_predictions = model.predict(preprocessed_image)

# Print raw predictions
print(f"Raw predictions for {image_path}: {raw_predictions}")

# Apply argmax to get predicted class labels
predicted_classes = [np.argmax(pred, axis=-1) for pred in raw_predictions]

# Print the predicted class labels for each attribute
print("Predicted classes:")
for idx, pred in enumerate(predicted_classes):
    print(f"Attribute {idx + 1}: {pred}")


In [ ]:
# Define your attribute mappings
attribute_mapping = {
    'attr_1': {0: 'black', 1: 'default', 2: 'multicolor', 3: 'white', 4: 'dummy'},
    'attr_2': {0: 'polo', 1: 'round', 2: 'dummy'},
    'attr_3': {0: 'printed', 1: 'solid', 2: 'dummy'},
    'attr_4': {0: 'default', 1: 'solid', 2: 'typography', 3: 'dummy'},
    'attr_5': {0: 'long sleeves', 1: 'short sleeves', 2: 'dummy'}
}

# Map predicted indices to labels
predicted_labels = {f'attr_{i + 1}': attribute_mapping[f'attr_{i + 1}'].get(pred[0], 'dummy') for i, pred in enumerate(predicted_classes)}

# Print the predicted labels
print("Predicted labels:")
for attr, label in predicted_labels.items():
    print(f"{attr}: {label}")


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image

# Preprocessing function: Resize images to (198, 198) and normalize pixel values to [0, 1]
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(198, 198))  # Resize to model's input size
    img_array = image.img_to_array(img)                       # Convert to array
    img_array = img_array / 255.0                             # Normalize pixel values
    return img_array

# Attribute mapping dictionary provided
attribute_mapping = {
    'attr_1': {0: 'black', 1: 'default', 2: 'multicolor', 3: 'white', 4: 'dummy'},
    'attr_2': {0: 'polo', 1: 'round', 2: 'dummy'},
    'attr_3': {0: 'printed', 1: 'solid', 2: 'dummy'},
    'attr_4': {0: 'default', 1: 'solid', 2: 'typography', 3: 'dummy'},
    'attr_5': {0: 'long sleeves', 1: 'short sleeves', 2: 'dummy'}
}

# Function to predict attributes for images in batches
def predict_attributes(image_folder, model, batch_size=128):
    predictions = []
    
    # Get all image files and sort them numerically based on the file name
    image_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.jpg')],
                         key=lambda x: int(x.split('.')[0]))  # Sorting by numerical part of the file name

    for i in range(0, len(image_files), batch_size):
        batch_files = image_files[i:i + batch_size]  # Process batch by batch
        batch_images = np.array([preprocess_image(os.path.join(image_folder, file)) for file in batch_files])

        # Predict attributes for the batch
        raw_predictions = model.predict(batch_images)

        # Extract predicted classes for each attribute
        predicted_classes = [np.argmax(pred, axis=-1) for pred in raw_predictions]

        # Map predicted indices to attribute labels
        for idx, file in enumerate(batch_files):
            predicted_labels = {
                f'attr_{j + 1}': attribute_mapping[f'attr_{j + 1}'][predicted_classes[j][idx]] 
                for j in range(len(predicted_classes))
            }
            #print(f"Image: {file}")
            #print("Predicted labels:", predicted_labels)
            #print("---")

    return predictions

# Example usage
image_folder = "/kaggle/working/sorted_images/Men_Tshirts"  # Update with your folder path
predict_attributes(image_folder, model, batch_size=128)


In [ ]:
import os
import csv
import numpy as np
from tensorflow.keras.preprocessing import image

# Preprocessing function: Resize images to (198, 198) and normalize pixel values to [0, 1]
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(198, 198))  # Resize to model's input size
    img_array = image.img_to_array(img)                       # Convert to array
    img_array = img_array / 255.0                             # Normalize pixel values
    return img_array

# Attribute mapping dictionary
attribute_mapping = {
    'attr_1': {0: 'black', 1: 'blue', 2: 'default', 3: 'green', 4: 'maroon', 5: 'multicolor', 6: 'navy blue', 7: 'peach', 8: 'pink', 9: 'red', 10: 'white', 11: 'yellow', 12: 'dummy'}, 'attr_2': {0: 'boxy', 1: 'default', 2: 'fitted', 3: 'regular', 4: 'dummy'}, 'attr_3': {0: 'crop', 1: 'regular', 2: 'dummy'}, 'attr_4': {0: 'default', 1: 'high', 2: 'round neck', 3: 'square neck', 4: 'stylised', 5: 'sweetheart neck', 6: 'v-neck', 7: 'dummy'}, 'attr_5': {0: 'casual', 1: 'party', 2: 'dummy'}, 'attr_6': {0: 'default', 1: 'printed', 2: 'solid', 3: 'dummy'}, 'attr_7': {0: 'default', 1: 'floral', 2: 'graphic', 3: 'quirky', 4: 'solid', 5: 'typography', 6: 'dummy'}, 'attr_8': {0: 'long sleeves', 1: 'short sleeves', 2: 'sleeveless', 3: 'three-quarter sleeves', 4: 'dummy'}, 'attr_9': {0: 'default', 1: 'puff sleeves', 2: 'regular sleeves', 3: 'sleeveless', 4: 'dummy'}, 'attr_10': {0: 'applique', 1: 'default', 2: 'knitted', 3: 'ruffles', 4: 'tie-ups', 5: 'waist tie-ups', 6: 'dummy'}
}

# Function to predict attributes for images in batches
def predict_attributes_and_save_to_csv(image_folder, model, batch_size=128):
    image_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.jpg')],
                         key=lambda x: int(x.split('.')[0]))  # Sorting by numerical part of the file name

    # CSV header
    header = ['id', 'Category', 'len', 'attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5', 'attr_6', 'attr_7', 'attr_8', 'attr_9', 'attr_10']

    # Open CSV file
    with open('predictions_WT.csv', mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=header)
        writer.writeheader()

        for i in range(0, len(image_files), batch_size):
            batch_files = image_files[i:i + batch_size]
            batch_images = np.array([preprocess_image(os.path.join(image_folder, file)) for file in batch_files])

            # Predict attributes for the batch
            raw_predictions = model.predict(batch_images)

            # Extract predicted classes for each attribute
            predicted_classes = [np.argmax(pred, axis=-1) for pred in raw_predictions]

            for idx, file in enumerate(batch_files):
                # Extract image id by removing leading zeros and file extension
                image_id = str(int(file.split('.')[0])) if file != '000000.jpg' else '0'

                # Map predicted classes to attribute labels
                predicted_labels = {
                    f'attr_{j + 1}': attribute_mapping[f'attr_{j + 1}'][predicted_classes[j][idx]]
                    for j in range(len(predicted_classes))
                }

                # Prepare a row with predicted labels and other columns
                row = {
                    'id': image_id,
                    'Category': 'Women Tops & Tunics',  # Fixed category value
                    'len': 10,                  # Fixed length value
                    'attr_1': predicted_labels['attr_1'],
                    'attr_2': predicted_labels['attr_2'],
                    'attr_3': predicted_labels['attr_3'],
                    'attr_4': predicted_labels['attr_4'],
                    'attr_5': predicted_labels['attr_5'],
                    'attr_6': predicted_labels['attr_6'],
                    'attr_7': predicted_labels['attr_7'],
                    'attr_8': predicted_labels['attr_8'],
                    'attr_9': predicted_labels['attr_9'],
                    'attr_10': predicted_labels['attr_10']
                }

                # Write the row to the CSV
                writer.writerow(row)

# Example usage
image_folder = "/kaggle/working/sorted_images/Women_Tops_&_Tunics"  # Update with your folder path
predict_attributes_and_save_to_csv(image_folder, model, batch_size=128)


In [7]:
category = "Sarees" 
num_attribute = 10
attribute_mapping = {
'attr_1': {0: 'default', 1: 'same as border', 2: 'same as saree', 3: 'solid', 4: 'dummy'}, 'attr_2': {0: 'default', 1: 'no border', 2: 'solid', 3: 'temple border', 4: 'woven design', 5: 'zari', 6: 'dummy'}, 'attr_3': {0: 'big border', 1: 'no border', 2: 'small border', 3: 'dummy'}, 'attr_4': {0: 'cream', 1: 'default', 2: 'green', 3: 'multicolor', 4: 'navy blue', 5: 'pink', 6: 'white', 7: 'yellow', 8: 'dummy'}, 'attr_5': {0: 'daily', 1: 'party', 2: 'traditional', 3: 'wedding', 4: 'dummy'}, 'attr_6': {0: 'default', 1: 'jacquard', 2: 'tassels and latkans', 3: 'dummy'}, 'attr_7': {0: 'default', 1: 'same as saree', 2: 'woven design', 3: 'zari woven', 4: 'dummy'}, 'attr_8': {0: 'default', 1: 'printed', 2: 'solid', 3: 'woven design', 4: 'zari woven', 5: 'dummy'}, 'attr_9': {0: 'applique', 1: 'botanical', 2: 'checked', 3: 'default', 4: 'elephant', 5: 'ethnic motif', 6: 'floral', 7: 'peacock', 8: 'solid', 9: 'dummy'}, 'attr_10': {0: 'no', 1: 'yes', 2: 'dummy'}}
category_path = category.replace(" ", "_")

In [8]:
import os
import csv
import numpy as np
from tensorflow.keras.preprocessing import image

# Preprocessing function: Resize images to (384, 384) and normalize pixel values to [-1, 1]
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(384, 384))  # Resize to model's input size
    img_array = image.img_to_array(img)                       # Convert to array
    img_array = img_array / 255.0 
    img_array = img_array * 2 - 1  # Normalize pixel values to range [-1, 1]
    return img_array

# Function to predict attributes for images in batches and save to CSV
def predict_attributes_and_save_to_csv(image_folder, model, batch_size=128, category=category, num_attribute=num_attribute):
    image_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.jpg')],
                         key=lambda x: int(x.split('.')[0]))  # Sort by numerical part of the file name

    # CSV header
    header = ['id', 'Category', 'len'] + [f'attr_{i+1}' for i in range(10)]

    # Open CSV file
    with open('v3s.csv', mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=header)
        writer.writeheader()

        for i in range(0, len(image_files), batch_size):
            batch_files = image_files[i:i + batch_size]
            batch_images = np.array([preprocess_image(os.path.join(image_folder, file)) for file in batch_files])

            # Predict attributes for the batch
            raw_predictions = model.predict(batch_images)

            # Remove the last element from each prediction in raw_predictions
            raw_predictions = [pred[:, :-1] for pred in raw_predictions]

            # Extract predicted classes for each attribute
            predicted_classes = [np.argmax(pred, axis=-1) for pred in raw_predictions]

            for idx, file in enumerate(batch_files):
                # Extract image id by removing leading zeros and file extension
                image_id = str(int(file.split('.')[0])) if file != '000000.jpg' else '0'

                # Map predicted classes to attribute labels up to num_attribute
                predicted_labels = {
                    f'attr_{j + 1}': attribute_mapping[f'attr_{j + 1}'][predicted_classes[j][idx]]
                    for j in range(num_attribute)
                }

                # Prepare a row with dynamic attributes and dummy values for the rest
                row = {
                    'id': image_id,
                    'Category': category,
                    'len': num_attribute
                }
                for j in range(1, 11):
                    attr_key = f'attr_{j}'
                    row[attr_key] = predicted_labels.get(attr_key, 'dummy')

                # Write the row to the CSV
                writer.writerow(row)

# Example usage
image_folder = f"/kaggle/input/sorted-test/sorted_images/{category_path}"  # Update with your folder path
predict_attributes_and_save_to_csv(image_folder, model, batch_size=128, category=category, num_attribute=num_attribute)


I0000 00:00:1731649844.269371     113 service.cc:145] XLA service 0x793644003350 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731649844.269437     113 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1731649844.269441     113 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


1/4 ━━━━━━━━━━━━━━━━━━━━ 51s 17s/step

I0000 00:00:1731649856.589801     113 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4/4 ━━━━━━━━━━━━━━━━━━━━ 18s 207ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 227ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 226ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 224ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 227ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 228ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 228ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 217ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 226ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 227ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 224ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 219ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 227ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 227ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 230ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 223ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 220ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 222ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 231ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 227ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 228ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 222ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 226ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 232ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 227ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━

In [16]:
import os
import csv
import numpy as np
from tensorflow.keras.preprocessing import image

# Preprocessing function: Resize images to (384, 384) and normalize pixel values to [-1, 1]
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(384, 384))  # Resize to model's input size
    img_array = image.img_to_array(img)                       # Convert to array
    img_array = img_array / 255.0 
    img_array = img_array * 2 - 1  # Normalize pixel values to range [-1, 1]
    return img_array

# Function to predict attributes for images in batches and save to CSV
def predict_attributes_and_save_to_csv(image_folder, model1, model2, batch_size=128, category=category, num_attribute=num_attribute):
    image_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.jpg')],
                         key=lambda x: int(x.split('.')[0]))  # Sort by numerical part of the file name

    # CSV header
    header = ['id', 'Category', 'len'] + [f'attr_{i+1}' for i in range(10)]

    # Open CSV file
    with open('inc$eff.csv', mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=header)
        writer.writeheader()

        for i in range(0, len(image_files), batch_size):
            batch_files = image_files[i:i + batch_size]
            batch_images = np.array([preprocess_image(os.path.join(image_folder, file)) for file in batch_files])

            # Predict attributes for the batch with both models
            raw_predictions1 = model1.predict(batch_images)
            raw_predictions2 = model2.predict(batch_images)

            # Average the predictions from both models
            averaged_predictions = [(pred1 + pred2) / 2 for pred1, pred2 in zip(raw_predictions1, raw_predictions2)]

            # Remove the last element from each prediction in averaged_predictions
            averaged_predictions = [pred[:, :-1] for pred in averaged_predictions]

            # Extract predicted classes for each attribute
            predicted_classes = [np.argmax(pred, axis=-1) for pred in averaged_predictions]

            for idx, file in enumerate(batch_files):
                # Extract image id by removing leading zeros and file extension
                image_id = str(int(file.split('.')[0])) if file != '000000.jpg' else '0'

                # Map predicted classes to attribute labels up to num_attribute
                predicted_labels = {
                    f'attr_{j + 1}': attribute_mapping[f'attr_{j + 1}'][predicted_classes[j][idx]]
                    for j in range(num_attribute)
                }

                # Prepare a row with dynamic attributes and dummy values for the rest
                row = {
                    'id': image_id,
                    'Category': category,
                    'len': num_attribute
                }
                for j in range(1, 11):
                    attr_key = f'attr_{j}'
                    row[attr_key] = predicted_labels.get(attr_key, 'dummy')

                # Write the row to the CSV
                writer.writerow(row)

# Example usage
image_folder = f"/kaggle/working/sorted_images/{category_path}"  # Update with your folder path
predict_attributes_and_save_to_csv(image_folder, model1, model2, batch_size=128, category=category, num_attribute=num_attribute)


4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 195ms/step


ValueError: Input 0 of layer "inception_multioutput" is incompatible with the layer: expected shape=(None, 299, 299, 3), found shape=(32, 384, 384, 3)

In [11]:
import os
import csv
import numpy as np
from tensorflow.keras.preprocessing import image

# Preprocessing function for model1 and model2: Resize to (384, 384) and normalize to [-1, 1]
def preprocess_image_model1_2(image_path):
    img = image.load_img(image_path, target_size=(384, 384))  # Resize to model's input size
    img_array = image.img_to_array(img)                       # Convert to array
    img_array = img_array / 255.0
    img_array = img_array * 2 - 1  # Normalize pixel values to range [-1, 1]
    return img_array

# Preprocessing function for model3: Resize to (299, 299) and normalize to [0, 1]
def preprocess_image_model3(image_path):
    img = image.load_img(image_path, target_size=(299, 299))  # Resize to 299x299 for model3
    img_array = image.img_to_array(img)                       # Convert to array
    img_array = img_array / 255.0  # Normalize to range [0, 1] for model3
    return img_array

# Function to predict attributes for images in batches and save to CSV
def predict_attributes_and_save_to_csv(image_folder, model1, model2, model3, batch_size=128, category=category, num_attribute=num_attribute):
    image_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.jpg')],
                         key=lambda x: int(x.split('.')[0]))  # Sort by numerical part of the file name

    # CSV header
    header = ['id', 'Category', 'len'] + [f'attr_{i+1}' for i in range(10)]

    # Open CSV file
    with open('ensemble3preds.csv', mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=header)
        writer.writeheader()

        for i in range(0, len(image_files), batch_size):
            batch_files = image_files[i:i + batch_size]

            # Preprocess images for model1 and model2 (384x384, [-1, 1])
            batch_images_model1_2 = np.array([preprocess_image_model1_2(os.path.join(image_folder, file)) for file in batch_files])

            # Preprocess images for model3 (299x299, [0, 1])
            batch_images_model3 = np.array([preprocess_image_model3(os.path.join(image_folder, file)) for file in batch_files])

            # Predict attributes for the batch with all three models
            raw_predictions1 = model1.predict(batch_images_model1_2)
            raw_predictions2 = model2.predict(batch_images_model1_2)
            raw_predictions3 = model3.predict(batch_images_model3)

            # Average the predictions from all three models
            averaged_predictions = [(pred1 + pred2 + pred3) / 3 for pred1, pred2, pred3 in zip(raw_predictions1, raw_predictions2, raw_predictions3)]

            # Remove the last element from each prediction in averaged_predictions
            averaged_predictions = [pred[:, :-1] for pred in averaged_predictions]

            # Extract predicted classes for each attribute
            predicted_classes = [np.argmax(pred, axis=-1) for pred in averaged_predictions]

            for idx, file in enumerate(batch_files):
                # Extract image id by removing leading zeros and file extension
                image_id = str(int(file.split('.')[0])) if file != '000000.jpg' else '0'

                # Map predicted classes to attribute labels up to num_attribute
                predicted_labels = {
                    f'attr_{j + 1}': attribute_mapping[f'attr_{j + 1}'][predicted_classes[j][idx]]
                    for j in range(num_attribute)
                }

                # Prepare a row with dynamic attributes and dummy values for the rest
                row = {
                    'id': image_id,
                    'Category': category,
                    'len': num_attribute
                }
                for j in range(1, 11):
                    attr_key = f'attr_{j}'
                    row[attr_key] = predicted_labels.get(attr_key, 'dummy')

                # Write the row to the CSV
                writer.writerow(row)

# Example usage
image_folder = f"/kaggle/working/sorted_images/{category_path}"  # Update with your folder path
predict_attributes_and_save_to_csv(image_folder, model1, model2, model3, batch_size=128, category=category, num_attribute=num_attribute)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 213ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 195ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 246ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 195ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 194ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 246ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 197ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 196ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 246ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 196ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 197ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 248ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 196ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 197ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 249ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 198ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 197ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 247ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 198ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 198ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 249ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 199ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 200ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 250ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 199ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 

In [18]:
import os
import csv
import numpy as np
from tensorflow.keras.preprocessing import image

# Preprocessing function for model1: Resize to (384, 384) and normalize to [-1, 1]
def preprocess_image_model1(image_path):
    img = image.load_img(image_path, target_size=(384, 384))  # Resize to model1's input size
    img_array = image.img_to_array(img)                       # Convert to array
    img_array = img_array / 255.0
    img_array = img_array * 2 - 1  # Normalize pixel values to range [-1, 1]
    return img_array

# Preprocessing function for model3: Resize to (299, 299) and normalize to [0, 1]
def preprocess_image_model3(image_path):
    img = image.load_img(image_path, target_size=(299, 299))  # Resize to 299x299 for model3
    img_array = image.img_to_array(img)                       # Convert to array
    img_array = img_array / 255.0  # Normalize to range [0, 1] for model3
    return img_array

# Function to predict attributes for images in batches and save to CSV
def predict_attributes_and_save_to_csv(image_folder, model1, model3, batch_size=128, category=category, num_attribute=num_attribute):
    image_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.jpg')],
                         key=lambda x: int(x.split('.')[0]))  # Sort by numerical part of the file name

    # CSV header
    header = ['id', 'Category', 'len'] + [f'attr_{i+1}' for i in range(10)]

    # Open CSV file
    with open('inc$eff_1.csv', mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=header)
        writer.writeheader()

        for i in range(0, len(image_files), batch_size):
            batch_files = image_files[i:i + batch_size]

            # Preprocess images for model1 (384x384, [-1, 1])
            batch_images_model1 = np.array([preprocess_image_model1(os.path.join(image_folder, file)) for file in batch_files])

            # Preprocess images for model3 (299x299, [0, 1])
            batch_images_model3 = np.array([preprocess_image_model3(os.path.join(image_folder, file)) for file in batch_files])

            # Predict attributes for the batch with both models
            raw_predictions1 = model1.predict(batch_images_model1)
            raw_predictions3 = model3.predict(batch_images_model3)

            # Average the predictions from model1 and model3
            averaged_predictions = [(pred1 + pred3) / 2 for pred1, pred3 in zip(raw_predictions1, raw_predictions3)]

            # Remove the last element from each prediction in averaged_predictions
            averaged_predictions = [pred[:, :-1] for pred in averaged_predictions]

            # Extract predicted classes for each attribute
            predicted_classes = [np.argmax(pred, axis=-1) for pred in averaged_predictions]

            for idx, file in enumerate(batch_files):
                # Extract image id by removing leading zeros and file extension
                image_id = str(int(file.split('.')[0])) if file != '000000.jpg' else '0'

                # Map predicted classes to attribute labels up to num_attribute
                predicted_labels = {
                    f'attr_{j + 1}': attribute_mapping[f'attr_{j + 1}'][predicted_classes[j][idx]]
                    for j in range(num_attribute)
                }

                # Prepare a row with dynamic attributes and dummy values for the rest
                row = {
                    'id': image_id,
                    'Category': category,
                    'len': num_attribute
                }
                for j in range(1, 11):
                    attr_key = f'attr_{j}'
                    row[attr_key] = predicted_labels.get(attr_key, 'dummy')

                # Write the row to the CSV
                writer.writerow(row)

# Example usage
image_folder = f"/kaggle/working/sorted_images/{category_path}"  # Update with your folder path
predict_attributes_and_save_to_csv(image_folder, model1, model3, batch_size=128, category=category, num_attribute=num_attribute)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 211ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 247ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 197ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 248ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 196ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 248ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 196ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 249ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 197ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 250ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 197ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 250ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 200ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 246ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 199ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 252ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 198ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 252ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 199ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 250ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 199ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 251ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 199ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 250ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 197ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 

In [22]:
import os
import csv
import numpy as np
from tensorflow.keras.preprocessing import image

# Preprocessing function for model1: Resize to (384, 384) and normalize to [-1, 1]
def preprocess_image_model1(image_path):
    img = image.load_img(image_path, target_size=(384, 384))  # Resize to model1's input size
    img_array = image.img_to_array(img)                       # Convert to array
    img_array = img_array / 255.0
    img_array = img_array * 2 - 1  # Normalize pixel values to range [-1, 1]
    return img_array

# Preprocessing function for model2: Resize to (384, 384) and normalize to [-1, 1]
def preprocess_image_model2(image_path):
    img = image.load_img(image_path, target_size=(384, 384))  # Resize to model2's input size
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = img_array * 2 - 1  # Normalize pixel values to range [-1, 1]
    return img_array

# Preprocessing function for model3: Resize to (299, 299) and normalize to [0, 1]
def preprocess_image_model3(image_path):
    img = image.load_img(image_path, target_size=(299, 299))  # Resize to 299x299 for model3
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize to range [0, 1] for model3
    return img_array

# Function to predict attributes for images in batches and save to CSV
def predict_attributes_and_save_to_csv(image_folder, model1, model2, model3, batch_size=128, category=category, num_attribute=num_attribute):
    image_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.jpg')],
                         key=lambda x: int(x.split('.')[0]))  # Sort by numerical part of the file name

    # CSV header
    header = ['id', 'Category', 'len'] + [f'attr_{i+1}' for i in range(10)]

    # Open CSV file
    with open('highest.csv', mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=header)
        writer.writeheader()

        for i in range(0, len(image_files), batch_size):
            batch_files = image_files[i:i + batch_size]

            # Preprocess images for each model
            batch_images_model1 = np.array([preprocess_image_model1(os.path.join(image_folder, file)) for file in batch_files])
            batch_images_model2 = np.array([preprocess_image_model2(os.path.join(image_folder, file)) for file in batch_files])
            batch_images_model3 = np.array([preprocess_image_model3(os.path.join(image_folder, file)) for file in batch_files])

            # Predict attributes for the batch with all three models
            raw_predictions1 = model1.predict(batch_images_model1)
            raw_predictions2 = model2.predict(batch_images_model2)
            raw_predictions3 = model3.predict(batch_images_model3)

            # Take the maximum value across predictions from all three models for each attribute
            combined_predictions = [np.maximum(np.maximum(pred1, pred2), pred3) 
                                    for pred1, pred2, pred3 in zip(raw_predictions1, raw_predictions2, raw_predictions3)]

            # Remove the last element from each prediction in combined_predictions
            combined_predictions = [pred[:, :-1] for pred in combined_predictions]

            # Extract predicted classes for each attribute
            predicted_classes = [np.argmax(pred, axis=-1) for pred in combined_predictions]

            for idx, file in enumerate(batch_files):
                # Extract image id by removing leading zeros and file extension
                image_id = str(int(file.split('.')[0])) if file != '000000.jpg' else '0'

                # Map predicted classes to attribute labels up to num_attribute
                predicted_labels = {
                    f'attr_{j + 1}': attribute_mapping[f'attr_{j + 1}'][predicted_classes[j][idx]]
                    for j in range(num_attribute)
                }

                # Prepare a row with dynamic attributes and dummy values for the rest
                row = {
                    'id': image_id,
                    'Category': category,
                    'len': num_attribute
                }
                for j in range(1, 11):
                    attr_key = f'attr_{j}'
                    row[attr_key] = predicted_labels.get(attr_key, 'dummy')

                # Write the row to the CSV
                writer.writerow(row)

# Example usage
image_folder = f"/kaggle/working/sorted_images/{category_path}"  # Update with your folder path
predict_attributes_and_save_to_csv(image_folder, model1, model2, model3, batch_size=128, category=category, num_attribute=num_attribute)


4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 197ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 208ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 237ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 192ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 193ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 245ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 195ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 194ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 245ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 194ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 195ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 247ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 196ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 196ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 247ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 209ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 196ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 249ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 197ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 197ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 248ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 198ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 198ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 250ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 210ms/step
4/4 ━━━━━━━━━━━━━━━━━━━

In [25]:
import pandas as pd
import glob

# List of CSV file paths
csv_files = ['/kaggle/working/v3m.csv', '/kaggle/working/v3s.csv', '/kaggle/working/v3k.csv', '/kaggle/working/v3w.csv', '/kaggle/working/v3wt.csv']  # Replace with actual file paths

# Read all CSV files into a single DataFrame
df_list = [pd.read_csv(file) for file in csv_files]

# Concatenate all DataFrames into one
combined_df = pd.concat(df_list, ignore_index=True)

# Convert 'id' column to numeric for proper sorting
combined_df['id'] = pd.to_numeric(combined_df['id'], errors='coerce')

# Sort the DataFrame by 'id' column in ascending order
sorted_df = combined_df.sort_values(by='id')

# Write the sorted DataFrame to a new CSV file
sorted_df.to_csv('bettercorr.csv', index=False)

print("Merged and sorted CSV file created successfully!")


Merged and sorted CSV file created successfully!


In [10]:
import pandas as pd

# Load the two CSV files
csv1_path = '/kaggle/input/only-dummy/only_dummy.csv'
csv2_path = '/kaggle/working/v3s.csv'

df1 = pd.read_csv(csv1_path)
df2 = pd.read_csv(csv2_path)

# Merge DataFrames by 'id', prioritizing rows from df2 when ids match
df_merged = pd.merge(df1, df2, on='id', how='left', suffixes=('', '_new'))

# Update columns in df1 with the values from df2 wherever the 'id' matched
for col in df2.columns:
    if col != 'id':  # Avoid updating the 'id' column itself
        df_merged[col] = df_merged[col + '_new'].combine_first(df_merged[col])
        df_merged.drop(columns=[col + '_new'], inplace=True)

# Save the result to a new CSV file
df_merged.to_csv('betttercorr_s.csv', index=False)

print("CSV updated successfully.")


CSV updated successfully.
